In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time_templates.templates.fit_muon_model_mean_traces import fit_Xmumax_lam
from time_templates.templates.universality.names import *
from time_templates.templates.muon_dsdt import MuondSdt, f_Xmumax_r_psi_theta
from time_templates.utilities.atmosphere import Atmosphere
import glob
from collections import defaultdict

from time_templates.utilities import plot as ttplot
from time_templates.utilities.fitting import plot_fit_curve
from time_templates.utilities.misc import histedges_equalN

atm = Atmosphere()

In [ ]:
from time_templates.templates.trace_templates import TraceTimeTemplate
ttt = TraceTimeTemplate(2100, 0, np.deg2rad(45), Xmax=800)

t = np.arange(0, 3000, 25/3)

f, ax = plt.subplots(1, figsize=(15, 7))

ttt.plot_wcd(t, 1, 1, 1, 1, ax=ax)

In [ ]:
dfs = []
for fl in glob.glob("/home/mart/auger/data/time_templates/mean_df/Xmumax/mean*Xmumax.pl"):
    df = pd.read_pickle(fl)
    df.index.rename([name + "_idx" for name in df.index.names], inplace=True)
    dfs.append(df)

dd = defaultdict(list)

ct2_bins = dfs[0].index.get_level_values(level=0).unique()
Xmumax_bins = dfs[0].index.get_level_values(level=1).unique()
print("Getting rs with most entries")  # 18.5 might not have 2000-2500 m
rs = [0]
for df in dfs:
    rs_ = df.index.get_level_values(level=2).unique()
    if len(rs_) > len(rs):
        rs = rs_
cps = df.index.get_level_values(level=3).unique()

print("Bins", ct2_bins, Xmumax_bins, rs, cps)

for ct2_bin in ct2_bins:
    ct2 = ct2_bin.mid
    print("at", ct2)
    for Xmumax_bin in Xmumax_bins:
        Xmumax = Xmumax_bin.mid
        for r in rs:
            for cp in cps:
                new_row = 0
                n_total = 0
                for df in dfs:
                    try:
                        row = df.loc[ct2, Xmumax, r, cp]
                        n = row["nstations"]
                        if n <= 0:
                            continue
                        new_row += row * n
                        n_total += n
                    except KeyError:
                        continue

                if n_total <= 0:
                    continue

                new_row /= n_total

                for key, val in new_row.items():
                    if key == "nstations":
                        continue
                    dd[key].append(val)

                dd["nstations"].append(n_total)
                dd["MCCosTheta_sq_bin_idx"].append(ct2_bin)
                dd["Xmumax_50_bin_idx"].append(Xmumax_bin)
                dd["MCr_round_idx"].append(r)
                dd["MCcospsi_round_idx"].append(cp)

df = pd.DataFrame(dd).set_index(dfs[0].index.names)
df.to_pickle('df_mean_merged_Xmumax.pl')

In [ ]:
df = pd.read_pickle('df_mean_merged_Xmumax.pl')

In [ ]:
from time_templates.templates.muon_dsdt import f_lambda_Xmumax_theta
def get_trace_fit(row, r, psi, theta, comp=eMUON, ax=None, fix_lambda=True):
    y = row[f'wcd_{comp}_trace_mean']
    yerr = np.sqrt(row[f'wcd_{comp}_trace_var']/row['nstations'])
    t0 = row['t0_wrt_pft_Rc_fit']
    t = np.linspace(t0, t0+599*25/3, 600)
    muon_dsdt = MuondSdt(r, psi, theta)
    m, _, _ = fit_Xmumax_lam(t, y, yerr, muon_dsdt, p0=[550, 40], fix_lambda=fix_lambda)
    print(muon_dsdt.Xmumax, muon_dsdt.lam, muon_dsdt.r, muon_dsdt.psi, muon_dsdt.theta)
    if ax is not None:
        pl = ax.errorbar(t, y, yerr=yerr, errorevery=3)
        ax.plot(t, muon_dsdt.ds_dt_wcd(t), '--', color=pl[0].get_color())
        ax.set_xlim([0, 2000])
    return m

In [ ]:
min_stations = 10
ax = None

dd = defaultdict(list)
#TODO: also em_mu component
for (ct2_bin, Xmumax_bin, r, cp), row in df.iterrows():
    dofit = True
    print()
    print(ct2_bin, Xmumax_bin, r, cp)

    if row["nstations"] < min_stations:
        dofit = False

    d_fit = {}
    theta = np.arccos(np.sqrt(ct2_bin.mid))
    psi = np.arccos(cp)
    success = True
    if dofit:
        try:
            m = get_trace_fit(row, r, psi, theta, ax=ax)
            if not (
                m.valid
                and m.accurate
                and not m.fmin.has_parameters_at_limit
                and m.fmin.has_posdef_covar
            ):
                pfit = [np.nan, np.nan]
                perr = [np.nan, np.nan]
                red_chi2 = np.nan
                success = False
            else:
                success = True
                pfit = m.values
                red_chi2 = m.fmin.reduced_chi2
                perr = m.errors * np.sqrt(red_chi2)
        except RuntimeError:
            pfit = [np.nan, np.nan]
            perr = [np.nan, np.nan]
            red_chi2 = np.nan
            success = False
    else:
        success = False
        pfit = [np.nan, np.nan]
        perr = [np.nan, np.nan]
        red_chi2 = np.nan

#     if success:
#         print("Success", ct2_bin, Xmumax_bin, r, cp, key)

    dd["Xmumax_fit"].append(pfit[0])
    dd['lam_fit'].append(pfit[1])
    dd['Xmumax_err'].append(perr[0])
    dd['lam_err'].append(perr[1])
    dd['red_chi2'].append(red_chi2)
    dd['success'].append(success)

df_fit = pd.DataFrame(dd)

In [ ]:
idx = df.index.names

df_join = df.reset_index().join(df_fit)
df_join = df_join.set_index(idx)
df_join.to_pickle('df_mean_merged_Xmumax_fitted_fixed_lambda.pl')

In [ ]:
df_join = pd.read_pickle('df_mean_merged_Xmumax_fitted_fixed_lambda.pl')

In [ ]:
# df_join = df_join.dropna()
df = df_join.loc[df_join['success']]

In [ ]:
ct2 = np.cos(np.deg2rad(45))**2
Xmumax_50 = 580
cp = 1
r = 800
psi = np.arccos(cp)
theta = np.arccos(np.sqrt(ct2))
row = df.loc[ct2, Xmumax_50, r, cp]
Xmax = row['MCXmax']
f, ax = plt.subplots(1)
t0 = row['t0_wrt_pft_Rc_fit']
print(t0)
t = np.linspace(t0, t0+599*25/3, 600)
ax.plot(t, row['wcd_muon_trace_mean'], 'k-')

from time_templates.templates.trace_templates import TraceTimeTemplate

tt = TraceTimeTemplate(r, psi, theta, Xmax, use_Xmumax=True, UUB=True)
tt.set_t0(t0)
ax.plot(t, tt.get_wcd_muon_trace_pdf(t), ls='--')
print(tt.muon_template.Xmumax, tt.muon_template.lam)
tt.muon_template.set_Xmumax_from_Xmumax_50(row['Xmumax_50'])
ax.plot(t, tt.muon_template.ds_dt_wcd(t), ls=':')
print(tt.muon_template.Xmumax, tt.muon_template.lam)
# m = get_trace_fit(row, r, psi, theta, fix_lambda=True, ax=ax)
ax.set_xlim([0, 1500])

In [ ]:
df_mean = df.groupby(["MCCosTheta_sq_bin_idx", "MCr_round_idx", "MCcospsi_round_idx"]).mean()
df_var = df.groupby(["MCCosTheta_sq_bin_idx", "MCr_round_idx", "MCcospsi_round_idx"]).apply(lambda x: np.mean(x**2))

In [ ]:
def Xmumax_r_func(r, a, b, r0=1700):
    r_ = r/r0
    return a * r_**-b

cps = df_mean.index.get_level_values(level=-1).unique()

ct2s = df_mean.index.get_level_values(level=0).unique()

dd = defaultdict(list)

for ct2_bin in ct2s:
    ct2 = ct2_bin.mid
    theta = np.arccos(np.sqrt(ct2))
    sin_theta = np.sin(theta)

    f, ax = plt.subplots(1, figsize=(7, 5))
    for cp in cps:

        df_mean_ = df_mean.loc[ct2, : , cp]
        df_var_ = df_var.loc[ct2, :, cp]
        cos_theta = df_mean_['MCCosTheta'].mean()

        x = df_mean_['MCr'].values
        y = df_mean_['Xmumax_fit'].values
        red_chi2 = df_mean_['red_chi2'].values
        yerr = np.sqrt(df_var_['Xmumax_err'].values*red_chi2)
        mask = np.isfinite(x*y*yerr) & (yerr > 0) & (red_chi2 < 100) & (x < 2100)
        if len(y[mask]) < 4:
            continue
        ax, (pfit, perr, chi2, ndof) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=Xmumax_r_func, p0=[600, 0.4], ax=ax, smoother_x=True)
        dd['cos_theta'].append(cos_theta)
        dd['cp'].append(cp)
        dd['p_a'].append(pfit[0])
        dd['p_b'].append(pfit[1])
#         dd['p_c'].append(pfit[2])
        dd['p_aerr'].append(perr[0])
        dd['p_berr'].append(perr[1])
#         dd['p_cerr'].append(perr[2])
        dd['red_chi2'].append(chi2/ndof)
        ax.legend(ncol=2)
#     break
df_fit_ab = pd.DataFrame(dd)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
sct_cut = 1.

pas = np.zeros((len(cps), 2))
pas_err = np.zeros((len(cps), 2))

pbs = np.zeros((len(cps), 2))
pbs_err = np.zeros((len(cps), 2))

pcs = np.zeros((len(cps), 2))
pcs_err = np.zeros((len(cps), 2))

for i, cp in enumerate(cps):
    df_ = df_fit_ab.query('cp == @cp')
    x = (1-df_['cos_theta']**2) - np.sin(np.deg2rad(50))**2
#     x = 1/df_['cos_theta']
    y = df_['p_a']
    yerr = df_['p_aerr']*np.sqrt(df_['red_chi2'])
    mask = np.isfinite(x*y*yerr) & (yerr > 0) #& (x > sct_cut)
    ax, (pfit, perr, _, _) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*x, ax=ax1, smoother_x=True)
    pas[i] = pfit
    pas_err[i] = perr
    y = df_['p_b']
    yerr = df_['p_berr']#*np.sqrt(df_['red_chi2'])
    mask = np.isfinite(x*y*yerr) & (yerr > 0) #& (x > sct_cut)
    ax, (pfit, perr, _, _) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*x, ax=ax2, smoother_x=True)
    pbs[i] = pfit
    pbs_err[i] = perr
#     y = df_['p_c']
#     yerr = df_['p_cerr']*np.sqrt(df_['red_chi2'])
#     mask = np.isfinite(x*y*yerr) & (yerr > 0) & (x > sct_cut)
#     ax, (pfit, perr, _, _) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-1.556), ax=ax3, smoother_x=True)
#     pcs[i] = pfit
#     pcs_err[i] = perr
ax1.legend(ncol=2, fontsize=8)
ax2.legend(ncol=2, fontsize=8)
# ax3.legend(ncol=2, fontsize=8)

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(10, 10))

plot_fit_curve(cps, pas[:, 0], yerr=pas_err[:, 0], func=lambda x, a, b: a+b*x, ax=axs[0, 0])
plot_fit_curve(cps, pas[:, 1], yerr=pas_err[:, 1], func=lambda x, a, b: a+b*x, ax=axs[0, 1])
plot_fit_curve(cps, pbs[:, 0], yerr=pbs_err[:, 0], func=lambda x, a, b: a+b*x, ax=axs[1, 0])
plot_fit_curve(cps, pbs[:, 1], yerr=pbs_err[:, 1], func=lambda x, a, b: a+b*x, ax=axs[1, 1])
# plot_fit_curve(cps, pcs[:, 0], yerr=pcs_err[:, 0], func=lambda x, a, b: a+b*x, ax=axs[2, 0], smoother_x=True)
# plot_fit_curve(cps, pcs[:, 1], yerr=pcs_err[:, 1], func=lambda x, a, b: a+b*x, ax=axs[2, 1])
for ax in axs.flatten():
    ax.legend()

In [ ]:
def f_Xmumax_r_psi_theta(r, psi, theta, a0=612.5, r0=1700, b=None):
    # TODO: the parametrization is very rough
    #     sec_theta = 1 / np.cos(theta) - 1.556
    sin_theta_sq = np.sin(theta) ** 2 - np.sin(np.deg2rad(50)) ** 2
    cos_psi = np.cos(psi)
    pa0 = a0 - 100.1 * cos_psi
    pa1 = 215 - 145 * cos_psi
    pb0 = 0.37 + 0.05 * cos_psi
    pb1 = 0.153 + 0.062 * cos_psi
    a = pa0 + pa1 * sin_theta_sq
    if b is None:
        b = pb0 + pb1 * sin_theta_sq
    return a * (r / r0) ** -b

In [ ]:
df['Xmumax_pred'] = f_Xmumax_r_psi_theta(df["MCr"],  np.arccos(df['MCcospsi']), df["MCTheta"])
df_ = df#.query('MCr > 700 & MCr < 1300')

ttplot.plot_hist(df_['Xmumax_pred']-df_["Xmumax_fit"], bins=np.linspace(-200, 200, 40), histtype='step', fit_norm=True);

In [ ]:
df['Xmumax_fit_diff'] = df['Xmumax_fit']-df['Xmumax_pred']+612.5
df_ = df.groupby("Xmumax_50_bin_idx").mean()
x = df_['Xmumax_50'].values
y = df_['Xmumax_fit_diff'].values
yerr = df.groupby("Xmumax_50_bin_idx")['Xmumax_fit_diff'].std()/np.sqrt(df.groupby("Xmumax_50_bin_idx")['Xmumax_fit_diff'].count())

f, ax = plt.subplots(1)

mask = np.isfinite(x*y*yerr) #& (x > 500) & (x < 650)
plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-600), ax=ax);
ax.legend()

In [ ]:
#a0 = 645 + 0.8 * (Xmumax_50-600)
#Xmumax_50 = (a0 - 645)/0.8 + 600

In [ ]:
((df['Xmumax_fit']-df['Xmumax_pred'])/0.8 + 600)

In [ ]:
df = pd.read_pickle('df_mean_merged_Xmumax_fitted.pl')
df = df.dropna()

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 7))

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors.append('k')
colors.append('gray')
ct2s = np.array([ct2_bin.mid for ct2_bin in df.index.get_level_values(level=0).unique()])

pfits = np.zeros((len(ct2s), 2))
perrs = np.zeros((len(ct2s), 2))

for i, ct2 in enumerate(ct2s):
    color = colors[i]
    df_ = df.loc[ct2]
    ax.scatter(df_['Xmumax_fit'], df_['lam_fit'], marker='.', alpha=0.2)
#     bins = histedges_equalN(df_['Xmumax_fit'], 10)
    bins = np.linspace(np.percentile(df_['Xmumax_fit'], 5), np.percentile(df_['Xmumax_fit'], 100), 10)
    
    ax, (x, y, yerr) = ttplot.plot_profile_1d(df_['Xmumax_fit'], df_['lam_fit'], bins=bins, ax=ax, color=color)
    mask = np.isfinite(x*y*yerr) & (yerr > 0)
    ax, (pfit, perr, chi2, ndf) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a + b * (x-600), ax=ax, ebar_kws=dict(color=color, ls=''))
    pfits[i] = pfit
    perrs[i] = perr#*np.sqrt(chi2/ndf)
#     ax.legend(ncol=2)
ax.set_ylim([0, 100])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ct2s_ = (ct2s[1:] + ct2s[:-1])/2
plot_fit_curve(ct2s_, pfits[:-1, 0], yerr=perrs[:-1, 0], func=lambda x, a, b: a +b *x, ax=ax1)
plot_fit_curve(ct2s_, pfits[:-1, 1], yerr=perrs[:-1, 1], func=lambda x, a, b: a +b *x, ax=ax2)

ax1.legend()
ax2.legend()

In [ ]:
def f_lambda_Xmumax_theta(Xmumax, theta):
    # average between proton iron at lgE18.5-20 costheta > 0.6
    p = [-4.5, 37.4, -0.024, -0.135]
    p00, p01, p10, p11 = p
    cos_theta_sq = np.cos(theta) ** 2
    sct_theta = 1 / np.cos(theta)
    p0 = p00 + p01 * sct_theta
    p1 = p10 + p11 * cos_theta_sq
    lam = p0 + p1 * (Xmumax - 600)
#     lam = min(70, max(lam, 5))
    return lam
df['lam_pred'] = f_lambda_Xmumax_theta(df['Xmumax_fit'].values, df['MCTheta'].values)

In [ ]:
df['rsinthetacospsi'] = df['MCcospsi']
df_ = df.query('MCCosTheta < 1 & MCr < 1900')
ax, (x, y, yerr) = ttplot.plot_profile_1d(df_['rsinthetacospsi'], df_['lam_fit']-df_['lam_pred'], bins=20, plot_n_entries=True);
mask = np.isfinite(x*y*yerr)
plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a + b*x, ax=ax);
ax.legend()

In [ ]:
ttplot.plot_hist(df['lam_fit']-(df['lam_pred'] + (-0.84 + 14.22*df['rsinthetacospsi'])), bins=np.linspace(-50, 50))

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))

ct2 = 0.4
Xmumax = 590
r = 500
cp = 1
fit_plot(ct2, Xmumax, r, cp, df_p, ax=ax)
fit_plot(ct2, Xmumax, r, cp, df_Fe, ax=ax);

In [ ]:


LXmu = 8000
r = 800l
psi = 0
theta = np.deg2rad(44)

atm.slant_depth_at_height((LXmu*np.cos(theta) + 1400 + r*np.cos(psi)*np.sin(theta)) , theta)- atm.slant_depth_at_height(((LXmu+1000)*np.cos(theta) + 1400 + r*np.cos(psi)*np.sin(theta)) , theta)

In [ ]:
from time_templates.datareader.get_data import fetch_MC_data_from_tree

In [ ]:
df_p = fetch_MC_data_from_tree(primary='proton', energy='19_19.5', dense=True)

In [ ]:
df_Fe = fetch_MC_data_from_tree(primary='iron', energy='19_19.5', dense=True)

In [ ]:
r = 600
df_p_ = df_p.query('MCr_round == @r & MCcospsi > 0.98 & Xmumax_50 > 550 & Xmumax_50 < 560 & MCCosTheta < 0.7 & MCCosTheta > 0.68')
df_Fe_ = df_Fe.query('MCr_round == @r & MCcospsi >-0.98 & Xmumax_50 > 550 & Xmumax_50 < 560 & MCCosTheta < 0.7 & MCCosTheta > 0.68')

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 5))
mean = np.zeros(2048)
for _, st in df_p_.iterrows():
    t0 = 0#st['t0_wrt_pft_Rc_fit']
    trace = st['wcd_muon_trace']
    t = np.linspace(t0, t0+(len(trace)-1)*25/3, len(trace))
    ax.plot(t, trace/trace.sum(), color='b', alpha=0.5)
    mean[:len(t)] += trace/trace.sum()
    
# ax.plot(mean/mean.sum(), color='b', lw=2)
mean = np.zeros(2048)

for _, st in df_Fe_.iterrows():
    t0 = 0#st['t0_wrt_pft_Rc_fit']
    trace = st['wcd_muon_trace']
    t = np.linspace(t0, t0+(len(trace)-1)*25/3, len(trace))
    ax.plot(t, trace/trace.sum(), color='r', alpha=0.5)
    mean[:len(t)] += trace/trace.sum()
# ax.plot(mean/mean.sum(), color='r', lw=2)

    
ax.set_xlim([0, 2000])